# Test RAGAs Evaluation

This notebook provides a minimal test setup for RAGAs evaluation.

## Features
- Small dataset subset for quick testing
- Cache clearing for rapid iterations
- Detailed error reporting

In [ ]:
import os
import sys
import json
import importlib
import asyncio
from pathlib import Path
from datasets import Dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ragas import evaluate
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextRecall,
    ContextPrecision,
    ContextEntityRecall,
    NoiseSensitivity
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_aws import ChatBedrockConverse, BedrockEmbeddings
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

# Add project root to path
project_root = Path("../..").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import utilities
from utils.aws.opensearch_utils import OpenSearchManager
from utils.notebook_utils.dataset_utils import load_labeled_dataset
from utils.notebook_utils.importable import notebook_to_module

In [ ]:
# Clear module cache to pick up changes
def clear_module_cache():
    """Clear imported module cache to pick up changes"""
    modules_to_clear = [
        m for m in sys.modules
        if m.startswith('utils.') or 
           m.startswith('rag_implementations.')
    ]
    for module in modules_to_clear:
        del sys.modules[module]
    print(f"Cleared {len(modules_to_clear)} modules from cache")

clear_module_cache()

In [ ]:
# OpenSearch Setup
OPENSEARCH_DOMAIN = "baseline-rag-benchmark-store"
INDEX_NAME = "originofcovid19dataset-benchmark"  # Fixed index name

print("Setting up OpenSearch...")
manager = OpenSearchManager(
    domain_name=OPENSEARCH_DOMAIN,
    cleanup_enabled=False,  # Don't clean up, we want to keep the index
    verbose=False
)

# Get endpoint and set environment variable
endpoint = manager.setup_domain()
os.environ['OPENSEARCH_HOST'] = endpoint
print(f"OpenSearch endpoint: {endpoint}")

# Create OpenSearch client
region = boto3.Session().region_name
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    'es',
    session_token=credentials.token
)

opensearch = OpenSearch(
    hosts=[{'host': endpoint, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

# Check index contents
print("\nChecking index...")
try:
    # Search for all documents
    response = opensearch.search(
        index=INDEX_NAME,
        body={
            "query": {"match_all": {}},
            "size": 1  # Just get one to check if index has content
        }
    )
    total_docs = response['hits']['total']['value']
    print(f"Found {total_docs} documents in index {INDEX_NAME}")
    
    if total_docs > 0:
        # Show sample document
        sample_doc = response['hits']['hits'][0]['_source']
        print("\nSample document:")
        print(f"Content length: {len(sample_doc['content'])} chars")
        print(f"Content preview: {sample_doc['content'][:200]}...")
        print(f"Metadata: {sample_doc.get('metadata', {})}")
    
except Exception as e:
    print(f"Error checking index: {str(e)}")
    raise

In [ ]:
# Load small subset of data
DATASET_DIR = project_root / "datasets/rag_evaluation/labeled/covid19_origin"
NUM_TEST_SAMPLES = 3  # Small subset for testing

print("Loading dataset...")
dataset, documents = load_labeled_dataset(DATASET_DIR)
test_examples = dataset.examples[:NUM_TEST_SAMPLES]
print(f"Using {len(test_examples)} test examples")

# Show example structure
example = test_examples[0]
print("\nExample structure:")
print(f"Query: {example.query}")
print(f"Reference answer: {example.reference_answer}")
print(f"Number of reference contexts: {len(example.reference_contexts)}")

In [ ]:
# Import RAG implementation
implementation_path = str(project_root / 'rag_implementations/baseline_rag/implementation.ipynb')
BaselineRAG = notebook_to_module(implementation_path).BaselineRAG

# Initialize RAG with same index name
rag = BaselineRAG(
    index_name=INDEX_NAME,
    chunk_size=500,
    chunk_overlap=50,
    enable_chunking=True
)

In [ ]:
# Test semantic search directly
print("Testing semantic search...")
example = test_examples[0]
print(f"\nQuery: {example.query}")

# Get relevant documents
docs = rag.semantic_search(example.query, k=3)
print(f"\nRetrieved {len(docs)} documents")

for i, doc in enumerate(docs, 1):
    print(f"\nDocument {i}:")
    print(f"Content length: {len(doc['content'])} chars")
    print(f"Content preview: {doc['content'][:200]}...")
    print(f"Metadata: {doc.get('metadata', {})}")

In [ ]:
# Generate answers and collect contexts
print("Generating answers...")
questions = []
answers = []
contexts = []
references = []

for example in test_examples:
    print(f"\nQuery: {example.query}")
    result = rag.query(example.query)
    
    questions.append(example.query)
    answers.append(result['response'])
    contexts.append([doc['content'] for doc in result['context']])
    references.append(example.reference_answer)  # Single string reference
    
    print(f"Retrieved {len(result['context'])} context documents")
    print(f"Answer: {result['response'][:100]}...")
    print(f"Reference: {example.reference_answer[:100]}...")

In [ ]:
# Create RAGAs dataset
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "reference": references  # Single string references
}

# Convert to dataset
eval_dataset = Dataset.from_dict(data)
print("Dataset structure:")
print(eval_dataset)

# Show first example
print("\nFirst example:")
example = eval_dataset[0]
print(f"Question: {example['question']}")
print(f"Answer: {example['answer'][:100]}...")
print(f"Number of contexts: {len(example['contexts'])}")
print(f"Reference: {example['reference'][:100]}...")

In [ ]:
# AWS config
config = {
    "region_name": "us-west-2",
    "llm": "anthropic.claude-3-sonnet-20240229-v1:0",
    "embeddings": "cohere.embed-english-v3",
    "temperature": 0.0
}

# Initialize evaluator models
evaluator_llm = LangchainLLMWrapper(ChatBedrockConverse(
    region_name=config["region_name"],
    base_url=f"https://bedrock-runtime.{config['region_name']}.amazonaws.com",
    model=config["llm"],
    temperature=config["temperature"],
))

evaluator_embeddings = LangchainEmbeddingsWrapper(BedrockEmbeddings(
    region_name=config["region_name"],
    model_id=config["embeddings"],
))

# Initialize metrics with wrapped models
metrics = [
    # Context retrieval metrics
    ContextPrecision(llm=evaluator_llm),
    ContextRecall(llm=evaluator_llm),
    ContextEntityRecall(llm=evaluator_llm),
    NoiseSensitivity(llm=evaluator_llm),
    
    # Answer generation metrics
    Faithfulness(llm=evaluator_llm),
    AnswerRelevancy(llm=evaluator_llm, embeddings=evaluator_embeddings)
]

# Run evaluation
print("Running evaluation...")
try:
    results = evaluate(
        dataset=eval_dataset,
        metrics=metrics
    )
    
    # Convert to pandas for better display
    df = results.to_pandas()
    print("\nResults:")
    print(df)
    
    # Group metrics by type
    print("\nContext Retrieval Metrics:")
    retrieval_metrics = ['context_precision', 'context_recall', 'context_entity_recall', 'noise_sensitivity_relevant']
    retrieval_scores = df[retrieval_metrics].mean()
    print(retrieval_scores)
    
    print("\nAnswer Generation Metrics:")
    generation_metrics = ['faithfulness', 'answer_relevancy']
    generation_scores = df[generation_metrics].mean()
    print(generation_scores)
    
    # Visualize results
    plt.figure(figsize=(12, 6))
    
    # Plot retrieval metrics
    plt.subplot(1, 2, 1)
    sns.barplot(x=retrieval_scores.index, y=retrieval_scores.values)
    plt.title('Context Retrieval Metrics')
    plt.xticks(rotation=45)
    plt.ylim(0, 1)
    
    # Plot generation metrics
    plt.subplot(1, 2, 2)
    sns.barplot(x=generation_scores.index, y=generation_scores.values)
    plt.title('Answer Generation Metrics')
    plt.xticks(rotation=45)
    plt.ylim(0, 1)
    
    plt.tight_layout()
    plt.show()
    
    # Analyze results
    print("\nAnalysis:")
    print("Context Retrieval:")
    print(f"- Precision: {retrieval_scores['context_precision']:.2f} - How many retrieved documents are relevant")
    print(f"- Recall: {retrieval_scores['context_recall']:.2f} - How many relevant documents were retrieved")
    print(f"- Entity Recall: {retrieval_scores['context_entity_recall']:.2f} - How well important entities are preserved")
    print(f"- Noise Sensitivity: {retrieval_scores['noise_sensitivity_relevant']:.2f} - Robustness against irrelevant data")
    
    print("\nAnswer Generation:")
    print(f"- Faithfulness: {generation_scores['faithfulness']:.2f} - How factual the answers are")
    print(f"- Relevancy: {generation_scores['answer_relevancy']:.2f} - How relevant the answers are to questions")
    
except Exception as e:
    print(f"Error during evaluation: {type(e).__name__}")
    print(f"Error details: {str(e)}")
    print("\nDataset contents:")
    for key, value in data.items():
        print(f"\n{key}:")
        print(f"Type: {type(value)}")
        print(f"Length: {len(value)}")
        print(f"First item: {value[0][:100]}...")
    raise